# Import Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import cm as c

from utils.val import *
from engine.train import *

import warnings
warnings.filterwarnings('ignore')

/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/seungmi/anaconda3/envs/pyDL/lib/python3

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


tf.enable_eager_execution() 

# Define/Import your own model

In [3]:
#################### edit this part ######################
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(None, None,3)))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(1))
    return model

In [4]:
model_base = create_model()

In [5]:
def ConvMax(filters, size, apply_batchnorm=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = keras.Sequential()
    result.add(
      keras.layers.Conv2D(filters, size, padding='same
                             ,kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(keras.layers.ReLU())
    result.add(keras.layers.MaxPool2D())
    
    return result

In [6]:
def Conv(filters, size, apply_batchnorm=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = keras.Sequential()
    result.add(
      keras.layers.Conv2D(filters, size, padding='same'
                             ,kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(keras.layers.ReLU())

    return result

In [75]:
def MCNN():
    inputs = tf.keras.layers.Input(shape=[None,None,3])

    mod1 = [
    ConvMax(16, 9, apply_batchnorm=False),
    ConvMax(32, 7),
    ConvMax(16, 7),
    Conv(8, 7),
    ]

    mod2 = [
    ConvMax(20, 7, apply_batchnorm=False),
    ConvMax(40, 5),
    ConvMax(20, 5),
    Conv(10, 5),
    ]
    
    mod3 = [
    ConvMax(24, 5, apply_batchnorm=False),
    ConvMax(48, 3),
    ConvMax(24, 3),
    Conv(12, 3),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    x = inputs

    for val1, val2, val3 in zip(mod1, mod2, mod3):
        x1 = val1(x)
        x2 = val2(x)
        x3 = val3(x)
        x = keras.layers.Concatenate()([x1,x2,x3])

    density_map = keras.layers.Conv2D(1, 1, padding='same', activation='linear')

    x = density_map(x)
    return tf.keras.Model(inputs=inputs, outputs=x)

In [76]:
model = MCNN()
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
sequential_125 (Sequential)     (None, None, None, 1 3904        input_24[0][0]                   
__________________________________________________________________________________________________
sequential_129 (Sequential)     (None, None, None, 2 2960        input_24[0][0]                   
__________________________________________________________________________________________________
sequential_133 (Sequential)     (None, None, None, 2 1824        input_24[0][0]                   
___________________________________________________________________________________________

# Train model

In [77]:
# define epochs and learning rates 
#############edit this if you want to change epochs or lr###############
fit(model, epochs=1, learning_rate = 0.02)

Learning started. It takes sometime.
Epoch: 1 loss = 73376080.00000000 Test MAE =  713.1232
Learning Finished!


# Visualize test outputs

In [ ]:
#############change test_input to check how your model works with a test data#######
generate_images(model, test_input = 'ShanghaiTech/part_A/test_data/images/IMG_100.jpg')